In [2]:
import csv

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

# Data Visualiation
import matplotlib.pyplot as plt
import seaborn as sns

# System
import os

# Deep Learning
"""
import tensorflow
from typing import Dict, List, Tuple, Sequence

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from sklearn.model_selection import train_test_split
from keras.layers import LSTM
"""

'\nimport tensorflow\nfrom typing import Dict, List, Tuple, Sequence\n\nfrom tensorflow.keras import models\nfrom tensorflow.keras import layers\nfrom tensorflow.keras import optimizers, metrics\nfrom tensorflow.keras.regularizers import L1L2\nfrom tensorflow.keras.layers.experimental.preprocessing import Normalization\nfrom tensorflow.keras.callbacks import EarlyStopping\nfrom tensorflow.keras.optimizers.schedules import ExponentialDecay\nfrom sklearn.model_selection import train_test_split\nfrom keras.layers import LSTM\n'

In [3]:
#to read file you saved
file_path_X_test="~/code/Agermita/intelligent_network_expansion/raw_data/X_test_all.csv"

In [4]:
df=pd.read_csv(file_path_X_test)

In [5]:
df.shape

(39999, 90)

In [16]:
import dask.dataframe as dd

In [17]:
dff= dd.from_pandas(df,npartitions=4)

: 

: 

In [6]:
df_reshaped=(list((" ".join((str(df[:][i]).strip("\n[]")).split())).split(" ")) for i in range(df))

TypeError: 'DataFrame' object cannot be interpreted as an integer

In [14]:
new_df=[]
for index, row in dff.iterrows():
    list_val=[]
    for index, value in row.items():
        new_list=list((" ".join((str(value).strip("\n[]")).split())).split(" "))
        
        list_val.append(new_list)
    new_df.append(list_val)

NameError: name 'dff' is not defined

In [12]:
X_test=np.array(new_df)
X_test

array([<generator object <genexpr> at 0x7f1887c26110>,
       <generator object <genexpr> at 0x7f1887c259a0>,
       <generator object <genexpr> at 0x7f1887c26180>, ...,
       <generator object <genexpr> at 0x7f1873b15230>,
       <generator object <genexpr> at 0x7f1873b152a0>,
       <generator object <genexpr> at 0x7f1873b15310>], dtype=object)

In [ ]:
X_test_changed=[]
for test in df:
  new_list=list((" ".join((data.strip("\n[]")).split())).split(" ") for data in test)
  print (len(new_list))
  X_test_changed.append(new_list)
#X_test_final=np.array(X_test_changed)
#X_test_final.shape

In [ ]:
print(len(read_X_train))

In [ ]:
def plot_history(history):

    fig, ax = plt.subplots(1,2, figsize=(20,7))
    # --- LOSS: MSE ---
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('MSE')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].legend(['Train', 'Validation'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)

    # --- METRICS:MAE ---

    ax[1].plot(history.history['mae'])
    ax[1].plot(history.history['val_mae'])
    ax[1].set_title('MAE')
    ax[1].set_ylabel('MAE')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Validation'], loc='best')
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)

    return ax

In [ ]:
def initialize_model(input_shape: tuple, output_length) -> models:
    #output_length = y_train.shape[1]
    #input_shape =(X.shape[1],X.shape[2])
    """
    Initialize the Neural Network with random weights
    """

    model = models.Sequential()
    ## 1.1 - Recurrent Layer
    model.add(layers.Masking(mask_value=-10, input_shape=input_shape))

    model.add(layers.LSTM(units=64,
                        activation='relu',
                        return_sequences = True,
                        kernel_regularizer=L1L2(l1=0.05, l2=0.05),
                        ))
    model.add(LSTM(64, activation='relu', return_sequences=True))
    model.add(LSTM(32, activation='sigmoid', return_sequences=False))
    ## 1.2 - Hidden Dense Layers

    model.add(layers.Dense(32, activation="relu", kernel_regularizer = L1L2(l1=0.05, l2=0.05)))
    model.add(layers.Dropout(rate=0.5))


    ## 1.2 - Predictive Dense Layers

    model.add(layers.Dense(output_length, activation='linear'))

    return model


def compile_model(model: models) -> models:
    """
    Compile the Neural Network
    """

    # 2 - Compiler
    # ======================
    initial_learning_rate = 0.01

    #lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.5)

    adam = optimizers.Adam(learning_rate=initial_learning_rate)
    #model.compile(loss='mse', optimizer="adam", metrics=["mae"])
    model.compile(loss='mse', optimizer=adam, metrics=['mae'])
    return model

def train_model(
        model: models,
        X: np.ndarray,
        y: np.ndarray,
        patience=2,
        validation_data=None, # overrides validation_split
        validation_split_rate=0.3
    ) -> Tuple[tensorflow.keras.Model, dict]:
    """
    Fit the model and return a tuple (fitted_model, history)
    """
     # $CHALLENGIFY_BEGIN
    es = EarlyStopping(monitor = "val_loss",
                      patience = patience,
                      mode = "min",
                      restore_best_weights = True)


    history = model.fit(X, y,
                        validation_split=validation_split_rate,
                        shuffle = False,
                        batch_size = 32,
                        epochs = 50,
                        callbacks = [es],
                        verbose = 1)

    return model, history

In [ ]:
input_shape =(X_train.shape[1],X_train.shape[2])
output_length = y_train.shape[1]

In [ ]:
input_shape =(90,26)
output_length = 30
model=initialize_model(input_shape, output_length)
model.summary()
model=compile_model(model)

In [ ]:
# train the model on one cell data
model, history=train_model(
    model,
    X_train,
    y_train,
    2,
    None, # don't use validation data, use validation split rate
    0.3
)

In [ ]:
"""
nwexamples = []
for row in read_X_train:
    nwrow = []
    for r in row:
        nwrow.append(eval(r))
    nwexamples.append(nwrow)
print(nwexamples)
"""